In [91]:
import pandas as pd
from sqlalchemy import create_engine


### Store CSV into DataFrame

In [92]:
from secrets import username, password


In [93]:
csv_file = "../kroger/kroger_milk_prices.csv"
kroger_data_df = pd.read_csv(csv_file)
kroger_data_df.head()


,Unnamed: 0,productId,categories,brand,description,size,price.regular,price.promo,Date,Store_Id,image
0,0,1111040101,['Dairy'],Kroger,Kroger® Vitamin D Whole Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
1,1,1111041600,['Dairy'],Kroger,Kroger® 2% Reduced Fat Milk,1/2 gal,1.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
2,2,1111041660,['Dairy'],Kroger,Kroger® 1% Lowfat Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
3,3,1111040601,['Dairy'],Kroger,Kroger® Vitamin D Whole Milk,1/2 gal,1.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
4,4,1111042315,['Dairy'],Kroger,Kroger® Fat Free Skim Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...


In [94]:
path = "../data/heb_clean.csv"
heb_data_df = pd.read_csv(path)
heb_milk = heb_data_df.drop(columns=['coupon', 'name' , 'price', 'uomSalePrice', 'type', 'simple_type'])
heb_milk = heb_milk.rename(columns= {'id':'product_id' , 'brand' : 'Brand'})
heb_milk['Store_Number'] = 1111111
heb_milk['Type_ID'] = 1
today = pd.datetime.now().date()
heb_data_df['Date'] = today
heb_data_df.head()
heb_milk.head()

<ipython-input-94-61ed49035a8a>:7: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime.now().date()


,Brand,category,features,product_id,image,size,Store_Number,Type_ID
0,Central Market,Dairy & Eggs/Milk,"[' ORGANIC', ' GO LOCAL']",931316,https://images.heb.com/is/image/HEBGrocery/prd...,1 gal,1111111,1
1,Central Market,Dairy & Eggs/Milk,"[' ORGANIC', ' GO LOCAL']",931317,https://images.heb.com/is/image/HEBGrocery/prd...,1 gal,1111111,1
2,H-E-B,Dairy & Eggs/Milk,"[' GO LOCAL', ' SELECT INGREDIENTS', ' Primo...",671076,https://images.heb.com/is/image/HEBGrocery/prd...,1/2 gal,1111111,1
3,H-E-B,Dairy & Eggs/Milk,"[' CARB AWARE', ' LOW SATURATED FAT', ' GO ...",314136,https://images.heb.com/is/image/HEBGrocery/prd...,1/2 gal,1111111,1
4,H-E-B,Dairy & Eggs/Milk,"[' GO LOCAL', ' SELECT INGREDIENTS']",314134,https://images.heb.com/is/image/HEBGrocery/prd...,1/2 gal,1111111,1


In [95]:
milk_df = kroger_data_df
milk_df = milk_df.drop(columns=['Unnamed: 0', 'description' , 'price.regular', 'price.promo', 'Date'])
features = 'none'
milk_df['features'] = features
milk_df = milk_df.rename(columns={"productId": "product_id", "brand": "Brand" , "categories" : "category" , 'Store_Id' : 'Store_Number'})
milk_df['Type_ID'] = 1
complete_milk = pd.concat([milk_df, heb_milk])
complete_milk

,product_id,category,Brand,size,Store_Number,image,features,Type_ID
0,1111040101,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
1,1111041600,['Dairy'],Kroger,1/2 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
2,1111041660,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
3,1111040601,['Dairy'],Kroger,1/2 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
4,1111042315,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
...,...,...,...,...,...,...,...,...
255,314131,Dairy & Eggs/Milk,Hill Country Fare,1 gal,1111111,https://images.heb.com/is/image/HEBGrocery/prd...,[' GO LOCAL'],1
256,314124,Dairy & Eggs/Milk,Hill Country Fare,1 gal,1111111,https://images.heb.com/is/image/HEBGrocery/prd...,[' GO LOCAL'],1
257,314128,Dairy & Eggs/Milk,Hill Country Fare,1 gal,1111111,https://images.heb.com/is/image/HEBGrocery/prd...,"[' CARB AWARE', ' LOW SATURATED FAT', ' GO ...",1
258,658779,Dairy & Eggs/Milk,H-E-B,1 gal,1111111,https://images.heb.com/is/image/HEBGrocery/prd...,"[' GO LOCAL', ' SELECT INGREDIENTS']",1


In [96]:
store_df = pd.DataFrame()
store_id = 3400312 , 1111111
store_df["Store_Number"] = store_id
zipcode = 77007 , 77007
store_df['Store_Zipcode'] = zipcode
name = "Kroger" , "HEB"
store_df['Store_Name'] = name
store_df.head()

,Store_Number,Store_Zipcode,Store_Name
0,3400312,77007,Kroger
1,1111111,77007,HEB


In [97]:
milk_type_df = pd.DataFrame()
milk_type = ['1%' , "2%", 'whole', 'skim', 'almond', 'other']
milk_type_df['Type'] = milk_type
milk_type_df.head()

,Type
0,1%
1,2%
2,whole
3,skim
4,almond


### Connect to local database

In [98]:
rds_connection_string = f"{username}:{password}@localhost:5432/milk_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [99]:
engine.table_names()

['Store', 'Milk', 'Milk_Type', 'Price_History']

Use drop duplicates to only add updated data to a dataframe that is then loaded into the database

In [100]:
temp_store = pd.read_sql_query('select * from "Store"', con=engine)
temp_store = temp_store.drop(columns = 'id')
#temp_store

In [101]:
db_store = pd.concat([temp_store, store_df]).drop_duplicates(keep=False)
db_store = db_store.reset_index(drop=True)   
#db_store.head()

### Use pandas to load csv converted DataFrame into database

In [102]:
db_store.to_sql(name='Store', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [103]:
pd.read_sql_query('select * from "Store"', con=engine).head()

,id,Store_Number,Store_Name,Store_Zipcode
0,1,3400312,Kroger,77007
1,2,1111111,HEB,77007


Same process as above for store table is followed for milk type.

In [104]:
temp_type = pd.read_sql_query('select * from "Milk_Type"', con=engine)
temp_type = temp_type.drop(columns = 'id')
db_type = pd.concat([temp_type, milk_type_df]).drop_duplicates(keep=False)
db_type = db_type.reset_index(drop=True)   
#db_type

In [105]:
db_type.to_sql(name='Milk_Type', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_location table

In [106]:
pd.read_sql_query('select * from "Milk_Type"', con=engine)

,id,Type
0,1,1%
1,2,2%
2,3,whole
3,4,skim
4,5,almond
5,6,other


Retrieves and merges Store data with milk data to set the Store ID then next cell deletes data not required in Milk Table

In [107]:
stores2_df = pd.read_sql_query('select * from "Store"', con=engine).head()
complete_milk = complete_milk.merge(stores2_df, on = 'Store_Number')
#complete_milk.head()

In [108]:
complete_milk = complete_milk.drop(columns = ['Store_Name' , 'Store_Zipcode', 'Store_Number'])
#complete_milk.head()

In [109]:
complete_milk = complete_milk.rename(columns ={'id' : 'Store_ID'})
#complete_milk.head()

Same process as above is followed to insert Milk data from dataframe into Milk table of database

In [110]:
temp_milk = pd.read_sql_query('select * from "Milk"', con=engine)
temp_milk = temp_milk.drop(columns = 'id')
#temp_milk

In [111]:
db_milk = pd.concat([temp_milk, complete_milk]).drop_duplicates(keep=False)
db_milk = db_milk.reset_index(drop=True)   
db_milk.head()

,product_id,Brand,features,size,category,image,Store_ID,Type_ID


In [112]:
db_milk.to_sql(name='Milk', con=engine, if_exists='append', index=False)

In [113]:
pd.read_sql_query('select * from "Milk"', con=engine)

,id,product_id,Brand,features,size,category,image,Store_ID,Type_ID
0,1,1.111040e+09,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
1,2,1.111042e+09,Kroger,none,1/2 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
2,3,1.111042e+09,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
3,4,1.111041e+09,Kroger,none,1/2 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
4,5,1.111042e+09,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
...,...,...,...,...,...,...,...,...,...
1255,1256,3.141310e+05,Hill Country Fare,[' GO LOCAL'],1 gal,Dairy & Eggs/Milk,https://images.heb.com/is/image/HEBGrocery/prd...,2,1
1256,1257,3.141240e+05,Hill Country Fare,[' GO LOCAL'],1 gal,Dairy & Eggs/Milk,https://images.heb.com/is/image/HEBGrocery/prd...,2,1
1257,1258,3.141280e+05,Hill Country Fare,"[' CARB AWARE', ' LOW SATURATED FAT', ' GO ...",1 gal,Dairy & Eggs/Milk,https://images.heb.com/is/image/HEBGrocery/prd...,2,1
1258,1259,6.587790e+05,H-E-B,"[' GO LOCAL', ' SELECT INGREDIENTS']",1 gal,Dairy & Eggs/Milk,https://images.heb.com/is/image/HEBGrocery/prd...,2,1


Milk data is pulled from database Milk table and merged with price history. Uneeded price history columns are dropped and renamed to coordinate with Price History Table. This sets the product ID in price history.

In [114]:
price_history_df = kroger_data_df[['Date', 'price.regular', 'price.promo', 'productId']].copy()
heb_price = heb_data_df[['price' , 'coupon', 'id', 'Date']]
heb_price =heb_price.rename(columns={'Date':'date', 'coupon':'saleprice', 'id':'product_id'})
price_history_df =price_history_df.rename(columns={'Date':'date', 'price.regular':'price', 'price.promo':'saleprice', 'productId': 'product_id'})
price_history_df = pd.concat([price_history_df, heb_price])
price_history_df['date'] = pd.to_datetime(price_history_df['date'])
price_history_df

,date,price,saleprice,product_id
0,2021-02-08,2.99,0.0,1111040101
1,2021-02-08,1.99,0.0,1111041600
2,2021-02-08,2.99,0.0,1111041660
3,2021-02-08,1.99,0.0,1111040601
4,2021-02-08,2.99,0.0,1111042315
...,...,...,...,...
255,2021-02-10,3.07,0.0,314131
256,2021-02-10,3.07,0.0,314124
257,2021-02-10,3.07,0.0,314128
258,2021-02-10,3.28,0.0,658779


In [115]:
milk2_df = pd.read_sql_query('select * from "Milk"', con=engine)
milk2_df = milk2_df.drop(columns = ['Brand' , 'size', 'image', 'Store_ID', 'Type_ID', 'category', 'features'])
merged_price = price_history_df.merge(milk2_df, on = 'product_id')
#merged_price

In [116]:
merged_price = merged_price.drop(columns = ['product_id'])
merged_price = merged_price.rename(columns = {'id' : 'product_id'})
#merged_price

Price History Dataframe is then added to Price History Table. Duplicates are not checked for in this instance because duplicates will trigger an error and stop the update process. 

In [117]:
temp_history = pd.read_sql_query('select * from "Price_History"', con=engine)
db_history = pd.concat([temp_history, merged_price]).drop_duplicates(keep=False)
db_history = db_history.reset_index(drop=True)   
db_history

,product_id,date,saleprice,price


In [118]:
db_history.to_sql(name='Price_History', con=engine, if_exists='append', index=False)

In [119]:
pd.read_sql_query('select * from "Price_History"', con=engine)

,product_id,date,saleprice,price
0,1,2021-02-08,0.0,2.99
1,2,2021-02-08,0.0,1.99
2,3,2021-02-08,0.0,2.99
3,4,2021-02-08,0.0,1.99
4,5,2021-02-08,0.0,2.99
...,...,...,...,...
1255,1256,2021-02-10,0.0,3.07
1256,1257,2021-02-10,0.0,3.07
1257,1258,2021-02-10,0.0,3.07
1258,1259,2021-02-10,0.0,3.28
